In [4]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns

def generate_report_assets(json_file='D:/College/Semester 7/Teknik Riset Operasi/Tubes/tubes_tro/results/optimized_routes.json'):
    print(f"📂 Membaca file: {json_file}...")
    
    # 1. LOAD DATA
    try:
        with open(json_file, 'r') as f:
            data = json.load(f)
        df = pd.DataFrame(data)
        print(f"✅ Data berhasil dimuat! Total: {len(df)} rute.")
    except FileNotFoundError:
        print(f"❌ Error: File '{json_file}' tidak ditemukan.")
        return

    # 2. PREPROCESSING
    # Pastikan tipe data benar
    df['Cost'] = df['Cost'].astype(float)
    df['Total_Dist'] = df['Total_Dist'].astype(float)
    df['Total_Time'] = df['Total_Time'].astype(float) # Asumsi dalam Jam
    
    # Set style visualisasi
    sns.set_theme(style="whitegrid")

    # ==============================================================================
    # GRAFIK 1: SCATTER PLOT (EFISIENSI)
    # ==============================================================================
    print("📈 Membuat Grafik 1: Efisiensi Rute...")
    plt.figure(figsize=(12, 7))
    
    scatter = sns.scatterplot(
        data=df, 
        x='Total_Dist', 
        y='Total_Time', 
        hue='Rit_Count', 
        palette='viridis', 
        s=100, 
        alpha=0.8,
        edgecolor='w'
    )
    
    # Garis Regresi (Tren)
    sns.regplot(data=df, x='Total_Dist', y='Total_Time', scatter=False, color='gray', line_kws={'linestyle':'--'})

    plt.title('Efisiensi Rute: Jarak vs Waktu Tempuh', fontsize=16, fontweight='bold')
    plt.xlabel('Total Jarak (km)', fontsize=12)
    plt.ylabel('Total Waktu (Jam)', fontsize=12)
    plt.legend(title='Jumlah Ritase', title_fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig('../figures/grafik_efisiensi_real.png', dpi=300)
    plt.close()

    # ==============================================================================
    # GRAFIK 2: BOX PLOT (ANALISIS BIAYA)
    # ==============================================================================
    print("💰 Membuat Grafik 2: Distribusi Biaya...")
    plt.figure(figsize=(10, 6))
    
    sns.boxplot(data=df, x='Rit_Count', y='Cost', palette='Set2')
    
    plt.title('Distribusi Biaya Operasional Berdasarkan Jumlah Ritase', fontsize=16, fontweight='bold')
    plt.xlabel('Jumlah Rit (Trip)', fontsize=12)
    plt.ylabel('Biaya Operasional (IDR)', fontsize=12)
    
    # Format sumbu Y agar tidak pakai notasi ilmiah (misal: 1e6)
    plt.ticklabel_format(style='plain', axis='y')
    
    # Tambahkan label angka rata-rata di grafik
    means = df.groupby(['Rit_Count'])['Cost'].mean()
    for i, val in enumerate(means):
        plt.text(i, val, f"Rp {val:,.0f}", horizontalalignment='center', color='black', weight='bold')

    plt.tight_layout()
    plt.savefig('../figures/grafik_biaya_ritase.png', dpi=300)
    plt.close()

    # ==============================================================================
    # GRAFIK 3: BAR CHART (BEBAN KERJA KECAMATAN)
    # ==============================================================================
    print("⚖️  Membuat Grafik 3: Beban Kecamatan...")
    # Agregasi data per kecamatan
    kec_summary = df.groupby('Kecamatan_ID')['Total_Load'].sum().reset_index()
    # Ambil Top 15 agar grafik tidak terlalu padat
    kec_summary = kec_summary.sort_values('Total_Load', ascending=False).head(15)

    plt.figure(figsize=(14, 7))
    sns.barplot(data=kec_summary, x='Kecamatan_ID', y='Total_Load', palette='magma')
    
    plt.title('Top 15 Kecamatan dengan Volume Sampah Tertinggi', fontsize=16, fontweight='bold')
    plt.xlabel('ID Kecamatan', fontsize=12)
    plt.ylabel('Total Beban Angkut (Ton)', fontsize=12)
    
    plt.tight_layout()
    plt.savefig('../figures/grafik_beban_kecamatan.png', dpi=300)
    plt.close()

    # ==============================================================================
    # TABEL RINGKASAN (SUMMARY)
    # ==============================================================================
    print("📊 Membuat Tabel Ringkasan...")
    summary = df.groupby('Rit_Count').agg({
        'Vehicle': 'count',
        'Total_Load': ['mean', 'sum'],
        'Total_Dist': 'mean',
        'Total_Time': 'mean',
        'Cost': 'mean'
    }).reset_index()

    # Rename kolom agar rapi
    summary.columns = ['Jumlah Rit', 'Total Armada', 'Avg Load (Ton)', 'Total Load (Ton)', 'Avg Jarak (km)', 'Avg Waktu (Jam)', 'Avg Biaya (IDR)']
    
    # Simpan ke CSV
    summary.to_csv('../dataset/tabel_analisis_operasional.csv', index=False)
    
    # Print ke layar
    print("\n" + "="*80)
    print("RINGKASAN PERFORMA OPERASIONAL")
    print("="*80)
    print(summary.to_string(index=False, formatters={
        'Avg Biaya (IDR)': 'Rp {:,.2f}'.format,
        'Avg Jarak (km)': '{:,.2f}'.format,
        'Avg Waktu (Jam)': '{:,.2f}'.format
    }))
    print("="*80)

    print("\n✅ Selesai! Semua file grafik (PNG) dan tabel (CSV) telah disimpan.")

if __name__ == "__main__":
    generate_report_assets()

📂 Membaca file: D:/College/Semester 7/Teknik Riset Operasi/Tubes/tubes_tro/results/optimized_routes.json...
✅ Data berhasil dimuat! Total: 151 rute.
📈 Membuat Grafik 1: Efisiensi Rute...
💰 Membuat Grafik 2: Distribusi Biaya...


C:\Users\USER\AppData\Local\Temp\ipykernel_22976\670566701.py:63: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=df, x='Rit_Count', y='Cost', palette='Set2')


⚖️  Membuat Grafik 3: Beban Kecamatan...


C:\Users\USER\AppData\Local\Temp\ipykernel_22976\670566701.py:91: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(data=kec_summary, x='Kecamatan_ID', y='Total_Load', palette='magma')


📊 Membuat Tabel Ringkasan...

RINGKASAN PERFORMA OPERASIONAL
 Jumlah Rit  Total Armada  Avg Load (Ton)  Total Load (Ton) Avg Jarak (km) Avg Waktu (Jam) Avg Biaya (IDR)
          1           102        5.325980            543.25          50.33            2.53   Rp 754,918.67
          2            38       11.695789            444.44          93.47            4.71 Rp 1,402,113.95
          3            11       17.991818            197.91         140.12            7.08 Rp 2,101,757.73

✅ Selesai! Semua file grafik (PNG) dan tabel (CSV) telah disimpan.


In [5]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Load Data
file_path = 'D:/College/Semester 7/Teknik Riset Operasi/Tubes/tubes_tro/results/optimized_routes.json'
with open(file_path, 'r') as f:
    data = json.load(f)

# Convert to DataFrame
df = pd.DataFrame(data)

# 2. Data Cleaning & Preparation
# Extract simplified status (memisahkan detail "+0.2h" jika ada)
df['Status_Category'] = df['Status'].apply(lambda x: x.split(' ')[0])

# 3. Visualization Setup
sns.set(style="whitegrid")

# --- CHART 1: Total Cost per Kecamatan (Saved Separately) ---
plt.figure(figsize=(12, 6))
cost_per_kec = df.groupby('Kecamatan_ID')['Cost'].sum().reset_index()
sns.barplot(x='Kecamatan_ID', y='Cost', data=cost_per_kec, palette='viridis')
plt.title('Total Operational Cost per Kecamatan', fontsize=14)
plt.xlabel('Kecamatan ID')
plt.ylabel('Total Cost (IDR)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('../figures/cost_per_kecamatan.png') # Simpan file 1
plt.close()
print("Saved: cost_per_kecamatan.png")

# --- CHART 2: Rit Count Distribution (Saved Separately) ---
plt.figure(figsize=(8, 6))
sns.countplot(x='Rit_Count', data=df, palette='magma')
plt.title('Distribution of Rit Counts (Trips per Vehicle)', fontsize=14)
plt.xlabel('Number of Rits')
plt.ylabel('Number of Vehicles')
for p in plt.gca().patches:
    plt.gca().annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')
plt.tight_layout()
plt.savefig('../figures/rit_count_distribution.png') # Simpan file 2
plt.close()
print("Saved: rit_count_distribution.png")

# --- CHART 3: Distance vs Time with Status (Saved Separately) ---
plt.figure(figsize=(10, 8))
sns.scatterplot(x='Total_Dist', y='Total_Time', hue='Status_Category', style='Status_Category', 
                data=df, s=100, palette={'AMAN': 'green', 'OVERTIME': 'red'})
plt.title('Correlation: Total Distance vs Total Time (Detecting Overtime)', fontsize=14)
plt.xlabel('Total Distance (km)')
plt.ylabel('Total Time (hours)')
plt.tight_layout()
plt.savefig('../figures/distance_vs_time.png') # Simpan file 3
plt.close()
print("Saved: distance_vs_time.png")

# --- CHART 4: Total Load Distribution per Kecamatan (Saved Separately) ---
plt.figure(figsize=(12, 6))
load_per_kec = df.groupby('Kecamatan_ID')['Total_Load'].sum().reset_index()
sns.barplot(x='Kecamatan_ID', y='Total_Load', data=load_per_kec, palette='coolwarm')
plt.title('Total Load (Volume) Handled per Kecamatan', fontsize=14)
plt.xlabel('Kecamatan ID')
plt.ylabel('Total Load')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('../figures/total_load_per_kecamatan.png') # Simpan file 4
plt.close()
print("Saved: total_load_per_kecamatan.png")

# --- Print Text Summary ---
print("\n--- Quick Insights ---")
print(f"Total Vehicles: {len(df)}")
print(f"Total Operational Cost: {df['Cost'].sum():,.0f}")

C:\Users\USER\AppData\Local\Temp\ipykernel_22976\1774882576.py:24: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='Kecamatan_ID', y='Cost', data=cost_per_kec, palette='viridis')


Saved: cost_per_kecamatan.png
Saved: rit_count_distribution.png


C:\Users\USER\AppData\Local\Temp\ipykernel_22976\1774882576.py:36: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(x='Rit_Count', data=df, palette='magma')


Saved: distance_vs_time.png


C:\Users\USER\AppData\Local\Temp\ipykernel_22976\1774882576.py:63: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='Kecamatan_ID', y='Total_Load', data=load_per_kec, palette='coolwarm')


Saved: total_load_per_kecamatan.png

--- Quick Insights ---
Total Vehicles: 151
Total Operational Cost: 153,401,369
